In [1]:
#IMPORTING LIBRARIE
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark import SparkConf


# DATA FRAME ORIGINAL - Project Tweets - Pandas

In [3]:
df = pd.read_csv('ProjectTweets.csv')
df.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


# EDA

In [4]:
df.shape

(1599999, 6)

In [5]:
df.dtypes

0                                                                                                                       int64
1467810369                                                                                                              int64
Mon Apr 06 22:19:45 PDT 2009                                                                                           object
NO_QUERY                                                                                                               object
_TheSpecialOne_                                                                                                        object
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    object
dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [7]:
df.isnull().sum()

0                                                                                                                      0
1467810369                                                                                                             0
Mon Apr 06 22:19:45 PDT 2009                                                                                           0
NO_QUERY                                                                                                               0
_TheSpecialOne_                                                                                                        0
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    0
dtype: int64

In [8]:
df.loc[-1] = df.columns
df.index = df.index + 1
df.sort_index(inplace=True)

In [9]:
df.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
new_columns = ["Index", "ID", "Date/Time", "Info", "User", "Tweet"]
df.columns = new_columns
df1 = df

In [11]:
df1.head(3)

,Index,ID,Date/Time,Info,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [12]:
df1.shape

(1600000, 6)

In [13]:
keywords = ["chatGPT", "AI", " artificial intelligence", "machine learning", "deep learning", "natural language processing", "neural networks", "chatbot", "OpeanAI", "Openai chat model", " neural networks" ]

In [14]:
filter = df1["Tweet"].str.contains('|'.join(keywords), case=False)
df2 = tweets_fil = df1[filter]

In [15]:
df2.head(5)

,Index,ID,Date/Time,Info,User,Tweet
7,7,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
10,10,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing
28,28,1467815199,Mon Apr 06 22:20:56 PDT 2009,NO_QUERY,abel209,ooooh.... LOL that leslie.... and ok I won't ...
30,30,1467815923,Mon Apr 06 22:21:07 PDT 2009,NO_QUERY,fatkat309,some1 hacked my account on aim now i have to ...
50,50,1467820863,Mon Apr 06 22:22:23 PDT 2009,NO_QUERY,tautao,Broadband plan 'a massive broken promise' http...


In [16]:
df2.shape

(205851, 6)

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205851 entries, 7 to 1599997
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Index      205851 non-null  object
 1   ID         205851 non-null  object
 2   Date/Time  205851 non-null  object
 3   Info       205851 non-null  object
 4   User       205851 non-null  object
 5   Tweet      205851 non-null  object
dtypes: object(6)
memory usage: 11.0+ MB


In [18]:
df2.to_parquet("/home/hduser/Desktop/TWEETS.parquet")

# Tratamento de dados (FIM)

In [19]:
spark_conf = SparkConf().setMaster("local[*]").setAppName("Tweets_Hadoop")

spark = SparkSession.builder.config(conf=spark_conf).config('spark.sql.session.timeZone', 'UTC').getOrCreate()

sc = spark.sparkContext

sc.setLogLevel('ERROR')



23/12/27 19:41:31 WARN Utils: Your hostname, BDS-2023 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/12/27 19:41:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 19:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
import os

file_path = "/CA4/TWEETS1.parquet"
if os.path.exists(file_path):
    print(f'O arquivo {file_path} existe.')
else:
    print(f'O arquivo {file_path} não existe.')


O arquivo /CA4/TWEETS1.parquet não existe.


In [21]:
tweets_spark = spark.read.parquet("hdfs://localhost:9000/CA4/TWEETS.parquet")


In [22]:
tweets_spark.printSchema()

root
 |-- Index: long (nullable = true)
 |-- ID: long (nullable = true)
 |-- Date/Time: string (nullable = true)
 |-- Info: string (nullable = true)
 |-- User: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [23]:
tweets_spark.summary().show()

+-------+-----------------+--------------------+--------------------+--------+--------------------+--------------------+
|summary|            Index|                  ID|           Date/Time|    Info|                User|               Tweet|
+-------+-----------------+--------------------+--------------------+--------+--------------------+--------------------+
|  count|           205851|              205851|              205851|  205851|              205851|              205851|
|   mean|773877.9395630821|2.0052840249757106E9|                null|    null|   9246832.818181818|                null|
| stddev|461638.8763636956|1.9429784188495928E8|                null|    null|3.0121803804319117E7|                null|
|    min|                7|          1467811594|Fri Apr 17 20:30:...|NO_QUERY|     000matthewkelly|                 ...|
|    25%|           377090|          1960372919|                null|    null|             10792.0|                null|
|    50%|           741136|     

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark import SparkConf


In [32]:
conf = SparkConf().set("spark.sql.legacy.timeParserPolicy", "LEGACY")
spark = spark.builder.config(conf=conf).getOrCreate()

In [25]:
tweets_spark = tweets_spark.withColumn("Date/Time", to_timestamp(tweets_spark["Date/Time"], "EEE MMM dd HH:mm:ss zzz yyyy"))

In [36]:
tweets_spark = tweets_spark.select("ID", "Date/Time", "User", "Tweet")

In [37]:
tweets_spark.show(truncate=True, n=5)

+----------+-------------------+--------------+--------------------+
|        ID|          Date/Time|          User|               Tweet|
+----------+-------------------+--------------+--------------------+
|1467811594|2009-04-07 05:20:03|          coZZ|@LOLTrish hey  lo...|
|1467812416|2009-04-07 05:20:16|erinx3leannexo|spring break in p...|
|1467815199|2009-04-07 05:20:56|       abel209|ooooh.... LOL  th...|
|1467815923|2009-04-07 05:21:07|     fatkat309|some1 hacked my a...|
|1467820863|2009-04-07 05:22:23|        tautao|Broadband plan 'a...|
+----------+-------------------+--------------+--------------------+
only showing top 5 rows



In [26]:
tweets_spark.printSchema()

root
 |-- Index: long (nullable = true)
 |-- ID: long (nullable = true)
 |-- Date/Time: timestamp (nullable = true)
 |-- Info: string (nullable = true)
 |-- User: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [38]:
tweets_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Date/Time: timestamp (nullable = true)
 |-- User: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [40]:
from pyspark.sql.functions import year, month, count, format_string, col

In [41]:
# Verifique a distribuição de tweets por mês
df = tweets_spark.groupBy(year("Date/Time").alias("year"), month("Date/Time").alias("month")).count() \
                 .orderBy(["year", "month"])

# Obtenha a porcentagem
df = df.withColumn("percentage", format_string("%.2f%%", ((col("count")/tweets_spark.count())*100)))

# Mostre o DataFrame resultante
df.show(truncate=False)

+----+-----+------+----------+
|year|month|count |percentage|
+----+-----+------+----------+
|2009|4    |12664 |6.15%     |
|2009|5    |68319 |33.19%    |
|2009|6    |124868|60.66%    |
+----+-----+------+----------+



In [42]:
#import modules
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover


In [44]:
#read csv file into dataFrame with automatically inferred schema
tweets_csv = spark.read.csv("hdfs://localhost:9000/CA4/tweets.csv", inferSchema=True, header=True)
tweets_csv.show(truncate=False, n=3)

+------+---------+---------------+---------------------------------+
|ItemID|Sentiment|SentimentSource|SentimentText                    |
+------+---------+---------------+---------------------------------+
|1038  |1        |Sentiment140   |that film is fantastic #brilliant|
|1804  |1        |Sentiment140   |this music is really bad #myband |
|1693  |0        |Sentiment140   |winter is terrible #thumbs-down  |
+------+---------+---------------+---------------------------------+
only showing top 3 rows



In [45]:
#select only "SentimentText" and "Sentiment" column, 
#and cast "Sentiment" column data into integer
data = tweets_csv.select("SentimentText", col("Sentiment").cast("Int").alias("label"))
data.show(truncate = False,n=5)

+---------------------------------+-----+
|SentimentText                    |label|
+---------------------------------+-----+
|that film is fantastic #brilliant|1    |
|this music is really bad #myband |1    |
|winter is terrible #thumbs-down  |0    |
|this game is awful #nightmare    |0    |
|I love jam #loveit               |1    |
+---------------------------------+-----+
only showing top 5 rows



In [47]:
#divide data, 70% for training, 30% for testing
dividedData = data.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1371 ; Testing data rows: 561


In [48]:
tokenizer = Tokenizer(inputCol="SentimentText", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False, n=5)

+----------------------------------+-----+----------------------------------------+
|SentimentText                     |label|SentimentWords                          |
+----------------------------------+-----+----------------------------------------+
|I adore cheese #favorite          |1    |[i, adore, cheese, #favorite]           |
|I adore cheese #thumbs-up         |1    |[i, adore, cheese, #thumbs-up]          |
|I adore cheese #toptastic         |1    |[i, adore, cheese, #toptastic]          |
|I adore classical music #brilliant|1    |[i, adore, classical, music, #brilliant]|
|I adore classical music #favorite |1    |[i, adore, classical, music, #favorite] |
+----------------------------------+-----+----------------------------------------+
only showing top 5 rows



In [49]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+----------------------------------+-----+----------------------------------------+-------------------------------------+
|SentimentText                     |label|SentimentWords                          |MeaningfulWords                      |
+----------------------------------+-----+----------------------------------------+-------------------------------------+
|I adore cheese #favorite          |1    |[i, adore, cheese, #favorite]           |[adore, cheese, #favorite]           |
|I adore cheese #thumbs-up         |1    |[i, adore, cheese, #thumbs-up]          |[adore, cheese, #thumbs-up]          |
|I adore cheese #toptastic         |1    |[i, adore, cheese, #toptastic]          |[adore, cheese, #toptastic]          |
|I adore classical music #brilliant|1    |[i, adore, classical, music, #brilliant]|[adore, classical, music, #brilliant]|
|I adore classical music #favorite |1    |[i, adore, classical, music, #favorite] |[adore, classical, music, #favorite] |
+-----------------------

In [50]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=3)

+-----+---------------------------+-------------------------------------------+
|label|MeaningfulWords            |features                                   |
+-----+---------------------------+-------------------------------------------+
|1    |[adore, cheese, #favorite] |(262144,[1689,100089,108624],[1.0,1.0,1.0])|
|1    |[adore, cheese, #thumbs-up]|(262144,[1689,88825,100089],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #toptastic]|(262144,[1689,42010,100089],[1.0,1.0,1.0]) |
+-----+---------------------------+-------------------------------------------+
only showing top 3 rows



In [51]:
lr = LogisticRegression(labelCol="label", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrainData)
print ("Training is done!")

Training is done!


In [52]:
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)

+-----+---------------------------+------------------------------------------+
|Label|MeaningfulWords            |features                                  |
+-----+---------------------------+------------------------------------------+
|1    |[adore, cheese, #bestever] |(262144,[1689,91011,100089],[1.0,1.0,1.0])|
|1    |[adore, cheese, #brilliant]|(262144,[1689,45361,100089],[1.0,1.0,1.0])|
+-----+---------------------------+------------------------------------------+
only showing top 2 rows



In [53]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
predictionFinal.show(n=4, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, 
      ", accuracy:", correctPrediction/totalData)

+------------------------------------+----------+-----+
|MeaningfulWords                     |prediction|Label|
+------------------------------------+----------+-----+
|[adore, cheese, #bestever]          |1.0       |1    |
|[adore, cheese, #brilliant]         |1.0       |1    |
|[adore, cheese, #loveit]            |1.0       |1    |
|[adore, classical, music, #bestever]|1.0       |1    |
+------------------------------------+----------+-----+
only showing top 4 rows



correct prediction: 551 , total data: 561 , accuracy: 0.982174688057041
